### Zagnieżdżanie:
###  Python układa instrukcje `try` na stosie. Po zgłoszeniu wyjątku Python poszukuje instrukcji `expect` pasującej do tego wyjątku. Sprawdza najpierw ostatnią instrukcję `try`. Jeśli tam jej nie znajdzie, to sprawdza wcześniejsze `try` aż dojdzie do najwyzszego poziomu programu. Po odnalezieniu odpowiedniego `expect` poszukiwanie zostaje zakonczone - życie wyjątku dobiega końca.
###  Jeśli zagnieżdżane są instrukcje `try` zawierające jedynie części `finally`, w momencie wystąpienia wyjątku wykonywany jest każdy blok `finally` po kolei aż trafi na `except` albo do najwyższego poziomu programu (i wyswietli błąd).

#### Zagnieżdżanie przebiegu sterowania:

In [2]:
def action2():
    print(1 + [])  # wywola wyjatek TypeError przez probe dodania liczby do listy

In [3]:
def action1():
    try:
        action2()
    except TypeError:   # przechwytuje TypeError z action2
        print('wewnetrzne try')

In [4]:
try:
    action1()
except TypeError:   # przechwytuje TypeError z action1 - jednak tutaj program nie dojdzie, bo wyjatek zostal obsluzony wewnatrz acion1
    print('zewnetrzne try')

wewnetrzne try


#### Zagnieżdżanie składniowe:

In [5]:
try:
    try:
        action2()
    except TypeError:
        print('wewnetrzne try')
except TypeError:
    print('zewnetrzne try')

wewnetrzne try


In [7]:
try:
    try:
        action2()
    finally:
        print('wewnetrzne try')
finally:
    print('zewnetrzne try')   # wykona wszystkie finally bo nie trafilo na expect


wewnetrzne try
zewnetrzne try


TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [8]:
def raise1(): raise IndexError
def noraise(): return
def raise2(): raise SyntaxError
    
for func in (raise1, noraise, raise2):
    print('\n', func, sep='')
    try:
        try:
            func()
        except IndexError:
            print('przechwycono IndexError')
    finally:
        print('wykonano finally')      


<function raise1 at 0x0000000005395AF0>
przechwycono IndexError
wykonano finally

<function noraise at 0x0000000005395D30>
wykonano finally

<function raise2 at 0x0000000005395940>
wykonano finally


SyntaxError: None (<string>)

### Zastosowanie wyjątków:
### 1. Funkcje mogą sygnalizować warunki za pomocą raise:
przykład: procedura wyszukiwania zgłasza wyjątek, kiedy dopasowanie zostanie odnalezione, zamiast zwracać opcję statusu, którą wywołujący musi zinterpretować:


    class Found(Exception): pass
    def searcher():
       if ...sukces...:
          raise Found()
       else:
          return
          
    try:
       searcher()
    except Found:                                # Wyjątek, jeśli element został odnaleziony
       ...sukces...
    else:                                        # else zwracane, kiedy element nie został odnaleziony
       ...porażka...
       
sygnalizowanie wyników bez zwracania wartości:

    class Failure(Exception): pass
    def searcher():
       if ...sukces...:
          return ...znaleziony element...
       else:
          raise Failure()
          
    try:
       item = searcher()
    except Failure:
       ...zgłoszenie...
    else:
       ...tutaj wykorzystanie elementu...
       
### 2. Zamykanie plików oraz połączeń z serwerem, np:
    myfile = open(r'C:\misc\script', 'w')
    try:
       ...przetworzenie myfile...
    finally:
       myfile.close()
       
albo
    
    with open(r'C:\misc\script', 'w') as myfile:
       ...przetworzenie myfile...

### 3. Debugowanie z wykorzystaniem zewnętrznych instrukcji try, np:

    try:
       ...wykonanie programu...
    except:                                      # Wszystkie nieprzechwycone wyjątki trafiają tutaj
       import sys
       print('nie przechwycono!', sys.exc_info()[0], sys.exc_info()[1])
       
### 4. Testowanie kodu wewnątrz tego samego procesu, np:

    import sys
    log = open('testlog', 'a')
    from testapi import moreTests, runNextTest, testName
    def testdriver():
       while moreTests():
          try:
             runNextTest()
          except:
             print('PORAŻKA', testName(), sys.exc_info()[:2], file=log)
          else:
             print('SUKCES', testName(), file=log)
    testdriver()

NameError: name 'oops' is not defined